In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER


In [2]:
import os

# os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax
import jax.numpy as jnp

In [3]:
jax.devices()

[CudaDevice(id=0)]

In [4]:
jax.devices('cpu')

[CpuDevice(id=0)]

In [5]:
o = jnp.ones(100)
o.device

CudaDevice(id=0)

In [6]:
from ernestogym.envs.single_agent.utils import parameter_generator

In [7]:
import os
print(os.getcwd())

/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER


In [ ]:
pack_options = "ernestogym/ernesto/data/battery/pack.yaml"
ecm = "ernestogym/ernesto/data/battery/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto/data/battery/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto/data/battery/models/thermal/r2c_thermal_pack.yaml"
bolun = "ernestogym/ernesto/data/battery/models/aging/bolun_pack.yaml"
world = "ernestogym/envs/single_agent/world_deg.yaml"
# world = "ernestogym/envs/single_agent/world_fading.yaml"

params = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    use_reward_normalization=True

)

params

In [ ]:
from ernestogym.ernesto.energy_storage.bess_degrading import BatteryEnergyStorageSystem

In [10]:
state = BatteryEnergyStorageSystem.get_init_state(models_config=params['models_config'],
                                                  battery_options=params['battery'],
                                                  input_var=params['input_var'])
print(state)

BessState(nominal_capacity=Array(60., dtype=float32, weak_type=True), c_max=Array(60., dtype=float32, weak_type=True), temp_ambient=Array(298.15, dtype=float32, weak_type=True), elapsed_time=Array(0., dtype=float32, weak_type=True), electrical_state=ElectricalModelState(r0_nominal=Array(10., dtype=float32, weak_type=True), r0=Array(10., dtype=float32, weak_type=True), rc=RCState(resistance_nominal=Array(10., dtype=float32, weak_type=True), resistance=Array(10., dtype=float32, weak_type=True), capacity=Array(10., dtype=float32, weak_type=True), i_resistance=Array(0., dtype=float32, weak_type=True)), ocv_potential=Array(10., dtype=float32, weak_type=True), is_active=Array(False, dtype=bool), v=Array(0., dtype=float32, weak_type=True), i=Array(0., dtype=float32, weak_type=True), v_rc=Array(0., dtype=float32, weak_type=True)), thermal_state=ThermalModelState(c_term=Array(138240, dtype=int32, weak_type=True), r_cond=Array(0.0061, dtype=float32, weak_type=True), r_conv=Array(0.1994, dtype=fl

In [11]:
# jax.tree.map(lambda a: jnp.array(a), state)

In [12]:
from time import time

t1 = time()
state = BatteryEnergyStorageSystem.step(state, 0.5, 1.)
print(time() - t1)

0.643690824508667


In [13]:
t1 = time()
for i in range(10000):
    state = BatteryEnergyStorageSystem.step(state, 0.5, 1.)

print(time() - t1)

4.15713357925415


In [14]:
# jax.make_jaxpr(BatteryEnergyStorageSystem.step)(state, 0.5, 1.)

In [15]:
state

BessState(nominal_capacity=Array(60., dtype=float32, weak_type=True), c_max=Array(60., dtype=float32, weak_type=True), temp_ambient=Array(298.15, dtype=float32, weak_type=True), elapsed_time=Array(0., dtype=float32, weak_type=True), electrical_state=ElectricalModelState(r0_nominal=Array(10., dtype=float32, weak_type=True), r0=Array(10., dtype=float32, weak_type=True), rc=RCState(resistance_nominal=Array(10., dtype=float32, weak_type=True), resistance=Array(10., dtype=float32, weak_type=True), capacity=Array(10., dtype=float32, weak_type=True), i_resistance=Array(-0.49999964, dtype=float32, weak_type=True)), ocv_potential=Array(10., dtype=float32, weak_type=True), is_active=Array(False, dtype=bool), v=Array(19.999996, dtype=float32, weak_type=True), i=Array(-0.5, dtype=float32, weak_type=True), v_rc=Array(-4.9999967, dtype=float32, weak_type=True)), thermal_state=ThermalModelState(c_term=Array(138240, dtype=int32, weak_type=True), r_cond=Array(0.0061, dtype=float32, weak_type=True), r_c

# Test parallel of states

In [16]:
import jax
import jax.numpy as jnp

In [17]:
state = BatteryEnergyStorageSystem.get_init_state(models_config=params['models_config'],
                                                  battery_options=params['battery'],
                                                  input_var=params['input_var'])

state_list = []

n_states = 400

for i in range(n_states):
    state = BatteryEnergyStorageSystem.step(state, 0.5, 1.)
    state_list.append(state)

states = jax.tree.map(lambda *vals: jnp.array(vals), *state_list)

In [18]:
states.nominal_capacity.shape

(400,)

In [19]:
# batched_step = jax.vmap(BatteryEnergyStorageSystem.step)
batched_step = jax.jit(jax.vmap(BatteryEnergyStorageSystem.step))   #, backend='cpu')

In [20]:
t1 = time()
states = batched_step(states, jnp.ones(n_states) * 0.5, jnp.ones(n_states))
print(time() - t1)

1.0683262348175049


In [21]:
# with jax.profiler.trace("/tmp/jax-trace", create_perfetto_link=True):
#     for i in range(1000):
#         states = batched_step(states, jnp.ones(n_states) * 0.5, jnp.ones(n_states))
#     jax.block_until_ready(states)

In [22]:
t1 = time()
for i in range(10000):
    states = batched_step(states, jnp.ones(n_states) * 0.5, jnp.ones(n_states))

jax.block_until_ready(states)
print(time() - t1)

23.343553066253662


In [23]:
states.nominal_capacity.device

CudaDevice(id=0)

## Jitted loop

In [24]:
from functools import partial

@partial(jax.jit, static_argnums=[0])
def maybe_loding(fun, states, act, times):
    for i in range(500):
        states = batched_step(states, act, times)
    return states

@partial(jax.jit, static_argnums=[0, 1])
def fused_loop(fun, num, states, act, times):
    def body_fun(i, states):
        return fun(states, act[i], times[i])
    return jax.lax.fori_loop(0, num, body_fun, states)

### single battery

In [29]:
t1 = time()

num = 10000

state_new = fused_loop(BatteryEnergyStorageSystem.step, num, state, jnp.ones(num) * 0.5, jnp.ones(num))

jax.block_until_ready(state_new)
print(time() - t1)

0.6907165050506592


### multiple batteries

In [30]:
t1 = time()

num = 10000

states_new = fused_loop(batched_step, num, states, jnp.ones((num, n_states)) * 0.5, jnp.ones((num, n_states)))

jax.block_until_ready(states_new)
print(time() - t1)

20.147693634033203
